In [5]:
import py2neo

graph = py2neo.Graph("bolt://localhost:7688", auth=("neo4j", "password_tuts"), name="mcu")

In [11]:
# 1: MCU network. Who are the 10 fastest in the MCU ? Are they related to each other ?

query = """MATCH (a:Superhero)
WHERE exists(a.Speed)
RETURN a
ORDER BY a.Speed DESC
LIMIT 10"""

data = graph.run(query).to_data_frame()
fastest10 = list(data["a"])
print(fastest10)

[Node('Superhero', Combat=56, Durability=84, Intelligence=75, Power=100, Speed=100, Strength=67, Total=482, id_=1009184, name='Black Bolt'), Node('Superhero', Combat=85, Durability=110, Intelligence=88, Power=100, Speed=100, Strength=85, Total=568, id_=1010764, name='Stardust'), Node('Superhero', Combat=56, Durability=60, Intelligence=63, Power=57, Speed=100, Strength=28, Total=364, id_=1009524, name='Quicksilver'), Node('Superhero', Combat=25, Durability=100, Intelligence=38, Power=100, Speed=100, Strength=60, Total=423, id_=1009477, name='Nova'), Node('Superhero', Combat=85, Durability=100, Intelligence=69, Power=100, Speed=92, Strength=100, Total=546, id_=1009664, name='Thor'), Node('Superhero', Combat=32, Durability=101, Intelligence=63, Power=100, Speed=84, Strength=100, Total=480, id_=1009592, name='Silver Surfer'), Node('Superhero', Combat=70, Durability=100, Intelligence=100, Power=100, Speed=83, Strength=100, Total=553, id_=1009312, name='Galactus'), Node('Superhero', Combat=8

In [15]:
query = """MATCH (a:Superhero)-[r:RELATION]->(b:Superhero)
WHERE a in $fastest10 AND b in $fastest10
RETURN a, r, b"""

data = graph.run(query, fastest10=fastest10).to_data_frame()
print(data)


Empty DataFrame
Columns: []
Index: []


In [17]:
# 2.  How many have collaborated atleast more than 10 times with another superhero?

query = """
    MATCH (a:Superhero)-[r:RELATION]->(b:Superhero)
    WITH a, count(r) as collabs
    WHERE collabs > 10
    RETURN count(distinct a) as num_superheroes
"""

data = graph.run(query).to_data_frame()
print(data)

   num_superheroes
0              670


In [26]:
# 3: Did the weakest of the MCU collaborated with the strongest ?
query = """ MATCH (a:Superhero)
    WHERE exists(a.Strength)
    RETURN a as weakest
    ORDER BY a.Strength ASC
    LIMIT 1 """ 

data = graph.run(query).to_data_frame()
weakest = list(data["weakest"])[0]
print(weakest)

query = """ MATCH (a:Superhero)
    WHERE exists(a.Strength)
    RETURN a as strongest
    ORDER BY a.Strength DESC
    LIMIT 1 """ 

data = graph.run(query).to_data_frame()
strongest = list(data["strongest"])[0]
print(strongest)

query = """MATCH (a:Superhero{id_:'1009393'})-[r:RELATION]-(b:Superhero{id_:'1009312'}) RETURN r,a,b"""
data = graph.run(query).to_data_frame()
print(data)

(_38:Superhero {Combat: 1, Durability: 1, Intelligence: 1, Power: 0, Speed: 1, Strength: 1, Total: 5, id_: 1009393, name: 'Lady Deathstrike'})
(_45:Superhero {Combat: 70, Durability: 100, Intelligence: 100, Power: 100, Speed: 83, Strength: 100, Total: 553, id_: 1009312, name: 'Galactus'})
Empty DataFrame
Columns: []
Index: []


In [57]:
# 5. Create a new db called "master". Create a graph from the M1 class with name and random age between 20 and 30, 
# Group of 3 that collab (undirected relation)
import tqdm
import numpy as np
import py2neo

graph = py2neo.Graph("bolt://localhost:7688", auth=("neo4j", "password_tuts"), name="master")
graph.delete_all()

names = {"0":"Alice",
        "1":"Bob",
        "2":"Charlie",
        "3":"David",
        "4":"Eve",
        "5":"Frank",
        "6":"Grace"}

try:
    graph.run("CREATE CONSTRAINT ON (:Person) ASSERT n.id_ IS UNIQUE")
except:
    print("Constraint already exists")

transaction_list = []
iteration = 0
for i in range(6):
    id_ = iteration
    name = names[str(i)]
    age = int(np.random.randint(20,30))
    transaction_list.append({"id_":id_,"name":name,"age":age})
    iteration += 1
    transaction = "UNWIND $json as data CREATE (n:Person) SET n = data"
    graph.run(transaction, json=transaction_list)
    transaction_list = []

graph.run(transaction, json=transaction_list)

Constraint already exists


(No data)

In [58]:
import random

student = list(range(6))

for _ in range(2): # 2 groups of 3
    selected_people = random.sample(student, 3) # select 3 random people
    id1, id2, id3 = sorted(selected_people) 
    transaction = """
    MATCH (a:Person{id_: $id1}), (b:Person{id_: $id2}), (c:Person{id_: $id3}) 
    CREATE (a)-[:RELATION]->(b), (b)-[:RELATION]->(c), (c)-[:RELATION]->(a),
           (a)-[:RELATION]->(c), (c)-[:RELATION]->(b), (b)-[:RELATION]->(a)
    """
    graph.run(transaction, id1=id1, id2=id2, id3=id3)

    for person in selected_people:
        student.remove(person)


In [61]:
# 6. Get the age for people YOU collaborate with

query = """MATCH (a:Person{name:'Alice'})-[:RELATION]-(b:Person) 
RETURN DISTINCT b.name AS Name, b.age AS Age
"""
data = graph.run(query).to_data_frame()
print(data)


      Name  Age
0  Charlie   27
1      Bob   27


In [62]:
# Get nodes with age > 25.

query = """MATCH (a:Person)
WHERE a.age > 25
RETURN a"""

data = graph.run(query).to_data_frame()
print(data)

                                          a
0    {'name': 'Alice', 'id_': 0, 'age': 28}
1      {'name': 'Bob', 'id_': 1, 'age': 27}
2  {'name': 'Charlie', 'id_': 2, 'age': 27}
3    {'name': 'David', 'id_': 3, 'age': 27}
4    {'name': 'Frank', 'id_': 5, 'age': 26}
